In [15]:
import pandas as pd

# Opening the file
sample_tl_raw = open("src/text data/Story_Tagalog.txt").read()

# Printing the Ilokano Raw Data
#sample_tl_raw

In [16]:
# Splitting the raw data into sentences
parsed_sp_tl_raw = sample_tl_raw.split("\n")

#parsed_sp_tl_raw 

In [17]:
dict_sm_tl = pd.DataFrame(parsed_sp_tl_raw, columns = ['Sentence'])

# Printing the first 5 rows of the DataFrame
dict_sm_tl.head()

,Sentence
0,"Nagbukas agad ng malamig na redhorse si Godo, ..."
1,Mayroon na ring pulutan na nakahain sa maliit ...
2,Imbes na alamano ay iniabot ni Godo ang bote n...
3,Sarap na sarap siya sa malamig na beer na duma...
4,"Ano pare, nakahain ka na ba? Tanong ng kumpare..."


## Viterbi Algorithm

In [18]:
# Opening the determiners file
tl_determiners = open("src/text data/TL_Determiners.txt").read()

tl_determiners

'ng\nang\nnang\nay\nmga\nna\nsa'

In [19]:
# Parsing the determiners file
parsed_tl_dtmn = tl_determiners.split("\n")

parsed_tl_dtmn

['ng', 'ang', 'nang', 'ay', 'mga', 'na', 'sa']

## Cleaning the Data

In [20]:
import string

def remove_punct(pText):
    text_nopumct = "".join([char for char in pText if char not in string.punctuation])
    return text_nopumct

cleaned_sp_tl = [remove_punct(word) for word in parsed_sp_tl_raw]

#cleaned_sp_tl[0:500]

In [21]:
import re

def tokenize(text):
    tokens = re.split('\W+', text.lower())
    return tokens


tokenized_sp_tl = [tokenize(word) for word in parsed_sp_tl_raw]

dict_sm_tl['Tokenized'] = tokenized_sp_tl
dict_sm_tl.head()

,Sentence,Tokenized
0,"Nagbukas agad ng malamig na redhorse si Godo, ...","[nagbukas, agad, ng, malamig, na, redhorse, si..."
1,Mayroon na ring pulutan na nakahain sa maliit ...,"[mayroon, na, ring, pulutan, na, nakahain, sa,..."
2,Imbes na alamano ay iniabot ni Godo ang bote n...,"[imbes, na, alamano, ay, iniabot, ni, godo, an..."
3,Sarap na sarap siya sa malamig na beer na duma...,"[sarap, na, sarap, siya, sa, malamig, na, beer..."
4,"Ano pare, nakahain ka na ba? Tanong ng kumpare...","[ano, pare, nakahain, ka, na, ba, tanong, ng, ..."


In [22]:
def check_dtmn(sentence, dtmn_list):
    """
    This function checks if the specific word in the sentence is a determiner, and extracts it.
    """
    text = [word for word in sentence if word in dtmn_list]
    
    return text


dict_sm_tl['Determiner'] = dict_sm_tl['Tokenized'].apply(lambda x: check_dtmn(x, parsed_tl_dtmn))

dict_sm_tl.head()

,Sentence,Tokenized,Determiner
0,"Nagbukas agad ng malamig na redhorse si Godo, ...","[nagbukas, agad, ng, malamig, na, redhorse, si...","[ng, na, sa]"
1,Mayroon na ring pulutan na nakahain sa maliit ...,"[mayroon, na, ring, pulutan, na, nakahain, sa,...","[na, na, sa, na]"
2,Imbes na alamano ay iniabot ni Godo ang bote n...,"[imbes, na, alamano, ay, iniabot, ni, godo, an...","[na, ay, ang, ng, na]"
3,Sarap na sarap siya sa malamig na beer na duma...,"[sarap, na, sarap, siya, sa, malamig, na, beer...","[na, sa, na, na, sa]"
4,"Ano pare, nakahain ka na ba? Tanong ng kumpare...","[ano, pare, nakahain, ka, na, ba, tanong, ng, ...","[na, ng]"


In [23]:
""" 
    Affixes
"""
PREFIX_SET = [
    'nakikipag', 'pakikipag',
    'pinakama', 'pagpapa',
    'pinagka', 'panganga',
    'makapag', 'nakapag',
    'tagapag', 'makipag',
    'nakipag', 'tigapag',
    'pakiki', 'magpa',
    'napaka', 'pinaka',
    'ipinag', 'pagka',
    'pinag', 'mapag',
    'mapa', 'taga',
    'ipag', 'tiga',
    'pala', 'pina',
    'pang', 'naka',
    'nang', 'mang',
    'sing',
    'ipa', 'pam',
    'pan', 'pag',
    'tag', 'mai',
    'mag', 'nam',
    'nag', 'man',
    'may', 'ma',
    'na', 'ni',
    'pa', 'ka',
    'um', 'in',
    'i', 'nagpa', 
    'magka', 'nagka',
    'ini',
    
]

INFIX_SET = [
    'um', 'in',
]

SUFFIX_SET = [
    'syon','dor',
    'ita', 'han',
    'hin', 'ing',
    'ang', 'ng',
    'an', 'in',
    'g',
]

PREPO_SET = [
    "sumasa", 'gitna',
    'ibabaw', 'ilalim',
    'itaas', 'ibabaw',
]

PER_PRONOUN = [
    'ako', 'ikaw', 'siya', 'kami', 'kayo', 'sila',
    'ko', 'akin', 'sakin', 'amin', 'atin', 'inyo',
    'kata', 'mo', 'kanila', 'kanya', 'namin', 'natin', 
    'ninyo', 'niya', 'kayoy', 'ikay', 'akoy', 'siyay', 'kamiy',
    'silay', 'inyoy', 'kanilay', 'kanyay', 'niyay'
]

noun_dtmn_list = ["ang", "ng", "mga", "si", "ay"] # Noun Determiners

adv_dtmn_list = ["nang"]

prepo_dtmn_list = ["sa"]

conj_list = ['at', 'o', 'saka', 'ngunit', 'datapwat']

vowels = ['a', 'e', 'i', 'o', 'u']



## Affixer Checker

In [24]:
def check_affixes(word, prev2_word, prev_word, next_word, prefix_list, infix_list, suffix_list, isDone, hasAffixes):
    """
    This function checks if the specific word in the sentence has an affix, and extracts it.
    """
    for prefix in prefix_list:
        if word.startswith(prefix) and not isDone:
            if word.startswith("mag"):
                if  word[3:5] == word[5:7] and not isDone:
                    """
                    verbs starting with "mag" always repeat the next 4 letters of the word e.g. maglalakad, maglalaro, magbibihis
                    """
                    hasAffixes = True
                    isDone = True

                if word[3] in (vowels):
                    """
                    verbs starting with "mag" and if the next letter is a vowel, the vowel is repeated e.g. magiikot, magaayos, maguusap
                    """
                    if word[3] == word[4] and not isDone:
                        hasAffixes = True
                        isDone = True
            else:
                hasAffixes = True
                isDone = True
                
    for infix in infix_list:
        if word.__contains__(infix) and not isDone:
            hasAffixes = True
            isDone = True
            
    for suffix in suffix_list:
        """
        words ending with 'ang' are adverbs and after the adverbs are the nouns 
        """
        if word.endswith(suffix) and not isDone and not word.endswith("ang") and not prev_word.endswith("ang"):
            hasAffixes = True
            isDone = True

    if len(word) >= 4:
        if word[:2] == word[2:4] and not isDone:
            """
            if the first four characters of a word is repeated, then it is a verb
            """
            hasAffixes = True
            isDone = True
    
    return hasAffixes
# end of check_affixes()

## Verb Tagger

In [25]:
def tag_verb(sentence, dtmn_list, prepo_list, pronoun_list):
    """
    This function tags if the specific word in the sentence is a verb, and extracts it.
    """
    
    """
    verb = [word for word in sentence if word in verb_list]
    """
    verb = []
    word = ""
    prev_word = ""
    prev2_word = ""
              
    for word in sentence:
        isDone = False
        hasAffixes = False
                
        try:
            next_word = sentence[sentence.index(word) + 1]
        except (ValueError, IndexError):
            next_word = None
            
        """
        getting the previous after the previous word
        """
        
        try:
            hasAffixes = check_affixes(word, prev2_word, prev_word, next_word, PREFIX_SET, INFIX_SET, SUFFIX_SET, isDone, hasAffixes)
        except (ValueError, IndexError):
            next_word = False
        
        if word not in (dtmn_list + prepo_list + pronoun_list + conj_list):
            
            if prev_word not in (noun_dtmn_list + adv_dtmn_list + prepo_dtmn_list): 
                if next_word in (noun_dtmn_list): 
                    """
                    if the previous word is not in the noun, adverb, and preposition determiner and 
                    the next word is a noun determiner
                    """
                    # hasAffixes = check_affixes(word, prev2_word, prev_word, next_word, PREFIX_SET, INFIX_SET, SUFFIX_SET, isDone, hasAffixes)
                    
                    if hasAffixes:
                        """
                        if the current word has an affix, then it is a verb
                        """
                        verb.append(word)
                        isDone = True
                
                if next_word in pronoun_list:
                    verb.append(word)
                    isDone = True

            if prev_word == "ay" and not isDone:
                if next_word in ("ng", "sa", "nang"):
                    """
                    if the previous word is 'ay' and the next word is 'ng' or 'sa', then it is a verb
                    """
                    verb.append(word)
                    isDone = True

            if word[:5] in ("magpa", "nagka") and not isDone or word[:4] in ("napa", "naka") and not isDone:
            # if hasAffixes and not isDone:
                """
                if the first five characters of a word start with "magpa" or "nagka", then it is a verb
                """
                verb.append(word)
                isDone = True
        
        if hasAffixes and prev_word == None and not isDone:
            if next_word in pronoun_list or (next_word in dtmn_list and next_word not in ('ng', 'mga')):
                verb.append(word)
                isDone = True
               
        prev_word = word
        
        try:
            prev2_word = sentence[sentence.index(word) - 1]
        except (ValueError, IndexError):
            prev2_word = None
        
    return verb
# end of function

dict_sm_tl['Verb'] = dict_sm_tl['Tokenized'].apply(lambda x: tag_verb(x, parsed_tl_dtmn, PREPO_SET, PER_PRONOUN))
dict_sm_tl.head(30)

,Sentence,Tokenized,Determiner,Verb
0,"Nagbukas agad ng malamig na redhorse si Godo, ...","[nagbukas, agad, ng, malamig, na, redhorse, si...","[ng, na, sa]",[nakaupo]
1,Mayroon na ring pulutan na nakahain sa maliit ...,"[mayroon, na, ring, pulutan, na, nakahain, sa,...","[na, na, sa, na]",[nakahain]
2,Imbes na alamano ay iniabot ni Godo ang bote n...,"[imbes, na, alamano, ay, iniabot, ni, godo, an...","[na, ay, ang, ng, na]",[]
3,Sarap na sarap siya sa malamig na beer na duma...,"[sarap, na, sarap, siya, sa, malamig, na, beer...","[na, sa, na, na, sa]",[]
4,"Ano pare, nakahain ka na ba? Tanong ng kumpare...","[ano, pare, nakahain, ka, na, ba, tanong, ng, ...","[na, ng]","[nakahain, tanong]"
5,"Oo pare, halika shot mo na muna para sakto mam...","[oo, pare, halika, shot, mo, na, muna, para, s...",[na],[shot]
6,Buti naman ata nakakuha sila ng permit na magp...,"[buti, naman, ata, nakakuha, sila, ng, permit,...","[ng, na, ang, ng, mga]","[nakakuha, magpasugal, nagka]"
7,"Ano ka ba naman pare, akala mo naman di moa la...","[ano, ka, ba, naman, pare, akala, mo, naman, d...","[na, sa, ng]","[akala, mayor]"
8,Nakatimbre na yan,"[nakatimbre, na, yan]",[na],[nakatimbre]
9,Kahit tingnan mo walang magrorondang pulis diy...,"[kahit, tingnan, mo, walang, magrorondang, pul...",[na],[tingnan]


In [26]:
def tag_noun(sentence, per_noun_list):
    """
    This function tags if the specific word in the sentence is a noun, and extracts it.
    """
    noun = []
    prev_word = ""
    prev2_word = ""
    adj_prefix = ["ika", "pinaka", "pang"]
    adj_suffix = ["ng"]
    
    
    for word in sentence:
        """
        marks if the word is already tagged
        """
        isDone = False
        
        """
        getting the next word
        """
        try:
            next_word = sentence[sentence.index(word) + 1]
        except (ValueError, IndexError):
            next_word = None
        
        
        if word in per_noun_list:
            """
            if the word is a personal pronoun, then it is a noun
            """
            noun.append(word)
            isDone = True
            
        if len(sentence) == 1:
            """
            if the sentence contains only one word, then it is a noun
            """
            noun.append(word)
            isDone = True
    
        if prev_word in (noun_dtmn_list) and word not in noun_dtmn_list and not isDone:
            """
            if the previous word is a determiner and the word is not a determiner, then it is a noun
            """
            isAdj = False
            isVerb = False
            
            if prev_word == 'ay' and next_word == 'ng':
                """
                if the previous word is 'ay' and the next word is 'ng', then it is a verb
                """
                isVerb = True
                
            if word.endswith("ng") and len(word.replace("ng", "")) > 3:
                """
                if the word ends with 'ng' and length of the word when 'ng' is removed is greater than 3, then it is an adjective
                """
                isAdj = True
            
            if not isVerb and not isAdj:
                for prefix in adj_prefix:
                    """
                    if the word is an adjective it has an adjective prefix
                    """
                    if not isDone:
                        isAdj = word.startswith(prefix)
                    if not isAdj and not isDone:
                        if next_word != 'ng':
                            noun.append(word)
                            isDone = True
                    if isAdj:  
                        isDone = True
                    
        if prev_word.startswith('pang') and prev_word.endswith('ng') and not isDone:
            """
            if the previous word is an adjective and if the next word is not an adjective
            then the word is a noun
            """
            noun.append(word)
            isDone = True
            
            
        if prev2_word == "ay" and prev_word.endswith("ang") and word not in noun_dtmn_list and not isDone:
            """
            if the previous previous word is "ay" and the previous word is "ang" 
            and the word is not a determiner then the word is a noun
            """
            noun.append(word)
            isDone = True
        
        
        """
        getting the previous word
        """
        prev_word = word
        
        
        
        """
        getting the previous after the previous word
        """
        try:
            prev2_word = sentence[sentence.index(word) - 1]
        except (ValueError, IndexError):
            prev2_word = None
        
    return noun
# end of function

dict_sm_tl['Noun'] = dict_sm_tl['Tokenized'].apply(lambda x: tag_noun(x, PER_PRONOUN))
dict_sm_tl.head(30)

,Sentence,Tokenized,Determiner,Verb,Noun
0,"Nagbukas agad ng malamig na redhorse si Godo, ...","[nagbukas, agad, ng, malamig, na, redhorse, si...","[ng, na, sa]",[nakaupo],"[malamig, godo, antokoy]"
1,Mayroon na ring pulutan na nakahain sa maliit ...,"[mayroon, na, ring, pulutan, na, nakahain, sa,...","[na, na, sa, na]",[nakahain],[]
2,Imbes na alamano ay iniabot ni Godo ang bote n...,"[imbes, na, alamano, ay, iniabot, ni, godo, an...","[na, ay, ang, ng, na]",[],"[iniabot, beer]"
3,Sarap na sarap siya sa malamig na beer na duma...,"[sarap, na, sarap, siya, sa, malamig, na, beer...","[na, sa, na, na, sa]",[],[siya]
4,"Ano pare, nakahain ka na ba? Tanong ng kumpare...","[ano, pare, nakahain, ka, na, ba, tanong, ng, ...","[na, ng]","[nakahain, tanong]",[kumpare]
5,"Oo pare, halika shot mo na muna para sakto mam...","[oo, pare, halika, shot, mo, na, muna, para, s...",[na],[shot],[mo]
6,Buti naman ata nakakuha sila ng permit na magp...,"[buti, naman, ata, nakakuha, sila, ng, permit,...","[ng, na, ang, ng, mga]","[nakakuha, magpasugal, nagka]","[sila, permit, higpit, pulis]"
7,"Ano ka ba naman pare, akala mo naman di moa la...","[ano, ka, ba, naman, pare, akala, mo, naman, d...","[na, sa, ng]","[akala, mayor]","[mo, bayan, natin]"
8,Nakatimbre na yan,"[nakatimbre, na, yan]",[na],[nakatimbre],[]
9,Kahit tingnan mo walang magrorondang pulis diy...,"[kahit, tingnan, mo, walang, magrorondang, pul...",[na],[tingnan],[mo]


## Tester

In [27]:
# temp_sen = dict_sm_tl['Tokenized'].array[14]
temp_sen = tokenize("maglalakad ako sa daan")
print(temp_sen)

tagged_sen = tag_verb(temp_sen, parsed_tl_dtmn, PREPO_SET, PER_PRONOUN) # verb tagger
# tagged_sen = tag_noun(temp_sen, PER_PRONOUN) # noun tagger
print(tagged_sen)


['maglalakad', 'ako', 'sa', 'daan']
['maglalakad']
